# Tweets Scrapping
### Steps
1. Set up tweepy authentication
2. Search Tweets
3. Append tweets info to a pandas dataframe

### Set up tweepy authentication

In [1]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API
import pandas as pd

# Consumer key authentication

api_key = 'zVCkUaVBASfQEqsDihlOXPk37'
api_key_secret = 'xEkxJOwfM5MvPINloFL3hDdVvyK0qRXAtFHcW4sgW4Wfet3iiR'
auth = OAuthHandler(api_key, api_key_secret)

access_token = '1456751370787115010-4cT91Wo3dfjFnoZmvjsF6ynBtabU2p'
access_token_secret = 'JRIPj8i5SynqHL1ROK8qujuzidNZQrMVJKQV13Pkp2hDJ'

# Access key authentication

auth.set_access_token(access_token, access_token_secret)

# Set up the API with the authentication handler
api = API(auth)

### Search Tweets

In [2]:
keywords = ['marriott']
limit=300
tweets = tweepy.Cursor(api.search,
                       q=keywords,
                       count=100,
                       tweet_mode='extended',
                       lang='en').items(limit)

### Append tweets info to a pandas dataframe

In [3]:
columns = ['User','Time','Text']
data = []
for tweet in tweets:
    if not tweet.retweeted and 'RT @' not in tweet.full_text:
        data.append([tweet.user.screen_name,tweet.created_at,tweet.full_text])
marriott_tweets = pd.DataFrame(data,columns=columns)
marriott_tweets.shape

(174, 3)

We got 157 tweets containing the keyword 'marriott'.

In [4]:
marriott_tweets['Hotel']='marriott'
marriott_tweets.head()

,User,Time,Text,Hotel
0,AmberSCosta,2022-02-28 01:30:24,Just posted a photo @ New York LaGuardia Airpo...,marriott
1,Jmart12323,2022-02-28 01:29:28,Thanks Marriott #MembersGetIt #RewardsPoints #...,marriott
2,MCYTanything,2022-02-28 01:27:58,James Marriott was not the impostor. https://t...,marriott
3,rebecca_suis,2022-02-28 01:26:34,@nancylevine @CarlosGSmith @Marriott @JerickFL...,marriott
4,NashvilleResist,2022-02-28 01:23:39,@nancylevine @MarriottIntl @Marriott also host...,marriott


### Same procedures for 'wyndham' and 'hilton'

In [5]:
keywords = ['wyndham']
limit=300
tweets = tweepy.Cursor(api.search,
                       q=keywords,
                       count=100,
                       tweet_mode='extended',
                       lang='en').items(limit)
columns = ['User','Time','Text']
data = []
for tweet in tweets:
    if not tweet.retweeted and 'RT @' not in tweet.full_text:
        data.append([tweet.user.screen_name,tweet.created_at,tweet.full_text])
wyndham_tweets = pd.DataFrame(data,columns=columns)
wyndham_tweets.shape

(207, 3)

We got 206 tweets containing the keyword 'wyndham'.

In [6]:
wyndham_tweets['Hotel']='wyndham'
wyndham_tweets.head()

,User,Time,Text,Hotel
0,MyClinicGroup,2022-02-28 01:20:28,WYNDHAM RESPIRATORY CLINIC IS NOW OPEN!\n\nWe ...,wyndham
1,summerbrennnan,2022-02-28 00:43:19,@mattRroth @stillgray BBC said this; they're t...,wyndham
2,SusanCast,2022-02-28 00:36:14,Friends ❤ en Nikki Beach Panama At Wyndham Gra...,wyndham
3,CaHomeoffice,2022-02-27 23:13:04,AWESOME - Austria is very proud. @SeppStraka i...,wyndham
4,T_Sardich,2022-02-27 23:03:49,@tr0jan04 @CaesarsRewards I’ve been requesting...,wyndham


In [7]:
keywords = ['hilton']
limit=300
tweets = tweepy.Cursor(api.search,
                       q=keywords,
                       count=100,
                       tweet_mode='extended',
                       lang='en').items(limit)
columns = ['User','Time','Text']
data = []
for tweet in tweets:
    if not tweet.retweeted and 'RT @' not in tweet.full_text:
        data.append([tweet.user.screen_name,tweet.created_at,tweet.full_text])
hilton_tweets = pd.DataFrame(data,columns=columns)
hilton_tweets.shape

(151, 3)

We got 156 tweets containing the keyword 'hilton'.

In [8]:
hilton_tweets['Hotel']='hilton'
hilton_tweets.head()

,User,Time,Text,Hotel
0,everylot_balt,2022-02-28 01:30:06,37 S HILTON ST https://t.co/TewriRdhrS,hilton
1,SpiviTech,2022-02-28 01:25:14,Arabella R. has joined the Spivi network from ...,hilton
2,SpiviTech,2022-02-28 01:25:07,R.B. has joined the Spivi network from Velocit...,hilton
3,H0USEWlVES,2022-02-28 01:23:31,Dina reassesses her connection with Jules aka....,hilton
4,malaysiandaily,2022-02-28 01:22:15,Check out B.S.B 1Kg/1000g Sleeping hilton Pill...,hilton


In [9]:
hotel_tweets = pd.concat([marriott_tweets, wyndham_tweets, hilton_tweets], ignore_index=True)

# Sentiment Prediction

### Steps
1. Textual Data Cleaning
2. Feature Engineering (Tfidf Vectorization)
3. Fit the mnb model on hotel reviews
4. Predict the sentiment of tweets

In [10]:
# Load hotel reviews
review = pd.read_csv('/Users/cynding/Desktop/Brandeis 2021 Fall/BUS 256A - Marketing Analytics/final project/marriott_hotel_reviews.csv')
review['Review']=review['Review'].astype(str)

#### 1. Textual Data Cleaning

In [14]:
# return the wordnet object value corresponding to the POS tag
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

'''
Part-Of-Speech (POS) tagging: assign a tag to every word to define 
if it corresponds to a noun, a verb etc. using the WordNet lexical database
'''

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def clean_text(text):
    # lower text
    text = text.lower()
    #
    text = text.replace("n't"," not")
    text = text.replace("i'm","i am")
    text = text.replace("i'd","i would")
    text = text.replace("i've","i have")
    text = text.replace("could've","could have")
    text = text.replace("we're","we are")
    text = text.replace("'ll"," will")
    text = text.replace("that's","that is")
    
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text: transform every word into their root form 
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

In [15]:
review['Review'] = review['Review'].apply(lambda x: clean_text(x))

In [16]:
hotel_tweets['Text'] = hotel_tweets['Text'].apply(lambda x: clean_text(x))

#### 2. Feature Engineering - TFIDF Vectorization

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
# For reviews text
tfidf = TfidfVectorizer(min_df = 5)
tfidf_result = tfidf.fit_transform(review['Review'].astype('U')).toarray()
tfidf_df = pd.DataFrame(tfidf_result,columns=tfidf.get_feature_names_out())
tfidf_df.columns = ["word_" + str(x) for x in tfidf_df.columns]
tfidf_df.index = review.index
review = pd.concat([review, tfidf_df], axis=1)

In [18]:
# For tiwtter text
tfidf = TfidfVectorizer()
tfidf_result = tfidf.fit_transform(hotel_tweets['Text'].astype('U')).toarray()
tfidf_df_tweets = pd.DataFrame(tfidf_result,columns=tfidf.get_feature_names_out())
tfidf_df_tweets.columns = ["word_" + str(x) for x in tfidf_df_tweets.columns]
tfidf_df_tweets.index = hotel_tweets.index
hotel_tweets = pd.concat([hotel_tweets, tfidf_df_tweets], axis=1)

#### 3. Fit the mnb model on hotel reviews

In [19]:
from sklearn.model_selection import train_test_split

# Models
from sklearn.naive_bayes import MultinomialNB

In [20]:
label = 'Total_Score'
features = [c for c in tfidf_df.columns]
X_train, X_test, y_train, y_test = train_test_split(review[features], review[label], test_size = 0.20, random_state = 42)

In [21]:
# train a naive bayes classifier
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

mnb_train_score = mnb.score(X_train,y_train)
mnb_test_score = mnb.score(X_test,y_test)
print(mnb_train_score,mnb_test_score)

0.6924891860007865 0.660377358490566


#### 4. Predict the sentiment of tweets

In [22]:
for i in features:
    if i not in tfidf_df_tweets.columns:
        tfidf_df_tweets[i] = 0

/var/folders/dl/l3zy8j197jn9h7zpll2v7q9w0000gn/T/ipykernel_41263/1269033137.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  tfidf_df_tweets[i] = 0


In [23]:
features = [c for c in tfidf_df.columns]
X_twitter = tfidf_df_tweets[features]
y_twitter_pred = mnb.predict(X_twitter)

hotel_tweets = pd.concat([hotel_tweets, pd.Series(y_twitter_pred)], axis=1)
hotel_tweets = hotel_tweets.rename(columns={0: "Predicted Score"})

In [24]:
hotel_tweets['Predicted Score'].describe()

count    532.000000
mean       4.994361
std        0.096873
min        3.000000
25%        5.000000
50%        5.000000
75%        5.000000
max        5.000000
Name: Predicted Score, dtype: float64

The problem with this sentiment prediction is it would be hard for the ML model to distinguish neutral comment from the positive comment.